# IQ Data Analysis in Octave
This Jupyter notebook walks you through reading raw I/Q data from a binary file (`test1.dat`), analyzing its power spectral density, visualizing samples in the time domain, plotting amplitude histograms, and saving the processed data for future use.

Each section includes background information and instructions. Feel free to modify parameters as needed to match your acquisition setup.


## 1. Initialization
We start by clearing the workspace, closing existing figures, and resetting the command window. This ensures a clean environment before running any analysis.

In [ ]:
% Clear workspace and figures
clear;
close all;
clc;

## 2. Specify Signal Characteristics
Here you define the key parameters of your signal acquisition:
- **Sampling frequency (`fs`):** How many samples per second were recorded.
- **Intermediate frequency (`fi`):** Any IF shift applied in front-end.
- **Data format flags:** Whether the data is complex I/Q and its bit-depth.
- **Duration (`secondOfData`):** How many seconds of data to read.
- **Center frequency (`fl1`):** The RF center frequency (e.g., 1575.42 MHz for GPS L1) used in labeling plots.

You will be prompted to enter the sampling frequency at runtime. Press **Enter** to accept the default shown in brackets.

In [ ]:
% Sampling and signal parameters
default_fs = 2.046e6;  % Default sampling frequency in Hz
prompt = sprintf('Enter sampling frequency in Hz [%g]: ', default_fs);
user_input = input(prompt, 's');
if isempty(user_input)
    fs = default_fs;
else
    fs = str2double(user_input);
end
fi = 0e6;           % Intermediate frequency
IsComplex = true;   % Flag: samples are I/Q
Is8Bits = false;    % 8-bit integer samples
Is16Bits = false;   % 16-bit integer samples
Is32BitsFloat = true; % 32-bit float samples
secondOfData = 1;   % Seconds of data to read
fl1 = 1575.42e6;    % Center frequency for plot labels

## 3. Read I/Q Samples from File
Open the binary file containing I/Q records and read the specified number of samples. Depending on your data format (8-bit, 16-bit, float), choose the correct `fread` and combine interleaved I and Q channels into complex-valued vectors.

In [ ]:
% Open and read binary I/Q data
fid = fopen('test1.dat', 'r');
if IsComplex
    if Is16Bits
        [raw, cnt] = fread(fid, 2*secondOfData*fs, 'int16');
    elseif Is8Bits
        [raw, cnt] = fread(fid, 2*secondOfData*fs, 'int8');
    elseif Is32BitsFloat
        [raw, cnt] = fread(fid, 2*secondOfData*fs, 'float32');
    end
    data = raw(1:2:end) + 1i*raw(2:2:end);
else
    if Is16Bits
        [data, cnt] = fread(fid, secondOfData*fs, 'int16');
    elseif Is8Bits
        [data, cnt] = fread(fid, secondOfData*fs, 'int8');
    elseif Is32BitsFloat
        [data, cnt] = fread(fid, secondOfData*fs, 'float32');
    end
end
fclose(fid);
% Remove the first 1e6 samples to avoid transients
if length(data) > 1e6
    data(1:1e6) = [];
end

## 4. Power Spectral Density
We compute the Power Spectral Density (PSD) using Welch's method (`pwelch`). This estimates the power distribution over frequency. We then shift and label the frequency axis relative to the center frequency for easy interpretation.

In [ ]:
% Compute and plot PSD
[pw, f] = pwelch(data, [], [], [], fs);
figure;
if IsComplex
    plot((f + fl1 - fi - fs/2)/1e6, 10*log10(fftshift(pw)));
else
    plot((f + fl1 - fi)/1e6, 10*log10(fftshift(pw)));
end
grid on;
axis tight;
set(gca, 'FontSize', 16);
xlabel('Frequency [MHz]');
ylabel('PSD [dBW/Hz]');
title('Power Spectral Density');

## 5. Time-Domain Visualization
Plot the real and imaginary parts of the first 100,000 samples. This helps inspect amplitude variations and verify signal quality over time.

In [ ]:
% Real component
figure;
plot(real(data(1:1e5)), 'r.');
set(gca, 'FontSize', 16);
xlabel('Sample ID'); ylabel('Amplitude');
title('Real Component');

% Imaginary component
figure;
plot(imag(data(1:1e5)), 'b.');
set(gca, 'FontSize', 16);
xlabel('Sample ID'); ylabel('Amplitude');
title('Imaginary Component');

## 6. Histogram of Real Component
Generate an amplitude histogram of the real part for the first 100,000 samples. This shows the distribution of sample values and can reveal clipping or quantization effects.

In [ ]:
% Histogram
figure;
hist(real(data(1:1e5)), 18);
set(gca, 'FontSize', 16);
xlabel('Amplitude'); ylabel('Occurrence');
title('Histogram of Real Component');

## 7. Save Processed Data
Save the cleaned and possibly truncated complex data vector to a MAT-file for future use.

In [ ]:
% Save to MAT-file
save('IQ_samples.mat', 'data');
disp('Saved IQ_samples.mat');